# Movie Recommendation System

### Import all dependencies

In [ ]:
import numpy as np
import pandas as pd
import difflib 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Data Collection and Preprocessing

In [ ]:
# load dataset (download the dataset from https://kaggle.com/datasets/muhammetgamal5/tmdbmoviescsv)


In [ ]:
# show dataset


In [ ]:
# number of rows and columns


In [ ]:
# missing values


In [ ]:
# selecting the relevant features for recommendation


In [ ]:
# replacing null values


In [ ]:
# combine all the selected features


In [ ]:
# Show combined features

### Feature Extraction

In [ ]:
# perform feature extraction

In [ ]:
# converting to numerical values


In [ ]:
#print feature vectors

### Cosine Similarity

In [ ]:
#calculate cosine similarity of feature vectors

In [ ]:
# show/print the results

In [ ]:
# input movie name 

In [ ]:
# creating a list with all the movie names


In [ ]:
#print movie list

In [ ]:
# finding the close match with input


In [ ]:
#print matched list

In [ ]:
# the movie name itself


In [ ]:
# finding index in dataset


In [ ]:
# calculate similarity score and display similarity list


In [ ]:
# sort this list to have the highest similarity score


In [ ]:
# code to print similar movies


### Recommandation System

In [ ]:
# take movie name


# finding the close match with input


# closest match - the searched movie most of the time


# index of closest match


# make a list with similarity socore and index, of that movie


# sort to get most similar movies at first


# print similar movies
